## Notebook to test out freesufer stats output without using freesurfer utilities

In [13]:
import pandas as pd
from freesurfer_stats import CorticalParcellationStats

In [75]:
ukbb_dkt_fields = "/home/nikhil/projects/brain_changes/brain-diff/metadata/UKBB_FS_DKT_Fields.csv"
UKBB_dkt_ct_fields = "/home/nikhil/projects/brain_changes/brain-diff/metadata/UKBB_DKT_CT_Fields.csv"

fs_output_dir = "/home/nikhil/projects/QPN_processing/test_data/fmriprep/output/freesurfer-6.0.1/"


In [121]:
stat_file = "aparc.DKTatlas.stats"
stat_measure = "average_thickness_mm" 
save_dir = './'

UKBB_dkt_ct_fields_df = pd.read_csv(UKBB_dkt_ct_fields)

hemispheres = ["lh", "rh"]

subject_id_list = ["sub-PD01369D713546","sub-NIMHANS001"]

hemi_stat_measures_dict = {}
for hemi in hemispheres:
    stat_measure_df = pd.DataFrame()
    for subject_id in subject_id_list:
        fs_stats_dir = f"{fs_output_dir}{subject_id}/stats/"
        stats = CorticalParcellationStats.read(f"{fs_stats_dir}{hemi}.{stat_file}").structural_measurements
        
        cols = ["subject_id"] + list(stats["structure_name"].values)
        vals = [subject_id] + list(stats[stat_measure].values)
        
        df = pd.DataFrame(columns=cols)
        df.loc[0] = vals
        stat_measure_df = pd.concat([stat_measure_df, df], axis=0)

    # replace columns names with ukbb field IDs
    field_df = UKBB_dkt_ct_fields_df[UKBB_dkt_ct_fields_df["hemi"]==hemi][["Field ID","roi"]]
    roi_field_id_dict = dict(zip(field_df["roi"], field_df["Field ID"]))
    stat_measure_df = stat_measure_df.rename(columns=roi_field_id_dict)
    
    hemi_stat_measures_dict[hemi] = stat_measure_df

# merge left and right dfs
stat_measure_LR_df = pd.merge(hemi_stat_measures_dict["lh"],hemi_stat_measures_dict["rh"], on="subject_id")

save_file = f"{stat_file.split('.')[1]}_{stat_measure.rsplit('_',1)[0]}.csv"

print(f"Saving stat measures here: {save_dir}/{save_file}")
stat_measure_LR_df.to_csv(f"{save_dir}/{save_file}")


Saving stat measures here: .//DKTatlas_average_thickness


In [120]:
stat_file = "aparc.DKTatlas.stats"
stat_measure = "average_thickness_mm" 


save_file

'DKTatlas_average_thickness'

## Read UKBB fields 
- This is based on UKBB showcase
- Only doing once and saving fieldIDs-ROIs csv

In [71]:
ukbb_dkt_fields_df = pd.read_csv(ukbb_dkt_fields)
ukbb_dkt_fields_df.head()

,Field ID,Description
0,27143,Area of caudalanteriorcingulate (left hemisphere)
1,27236,Area of caudalanteriorcingulate (right hemisph...
2,27144,Area of caudalmiddlefrontal (left hemisphere)
3,27237,Area of caudalmiddlefrontal (right hemisphere)
4,27145,Area of cuneus (left hemisphere)


### Grab rows with specific stat measure of interest

In [72]:
stat_measure = "Mean thickness" 
ukbb_dkt_measure_df = ukbb_dkt_fields_df[ukbb_dkt_fields_df["Description"].str.startswith(stat_measure)].copy()
ukbb_dkt_measure_df["Description"] = ukbb_dkt_measure_df["Description"].str.replace("Mean thickness","Mean_thickness")
print(f"number of fields with {stat_measure}: {len(ukbb_dkt_measure_df)}")

number of fields with Mean thickness: 62


In [73]:
ukbb_dkt_measure_df[["metric","roi","hemi"]] = ukbb_dkt_measure_df["Description"].str.split(" ", expand=True)[[0,2,3]]
ukbb_dkt_measure_df["hemi"] = ukbb_dkt_measure_df["hemi"].replace({"(left":"lh", "(right":"rh"})
ukbb_dkt_measure_df.head()

,Field ID,Description,metric,roi,hemi
62,27174,Mean_thickness of caudalanteriorcingulate (lef...,Mean_thickness,caudalanteriorcingulate,lh
63,27267,Mean_thickness of caudalanteriorcingulate (rig...,Mean_thickness,caudalanteriorcingulate,rh
64,27175,Mean_thickness of caudalmiddlefrontal (left he...,Mean_thickness,caudalmiddlefrontal,lh
65,27268,Mean_thickness of caudalmiddlefrontal (right h...,Mean_thickness,caudalmiddlefrontal,rh
66,27176,Mean_thickness of cuneus (left hemisphere),Mean_thickness,cuneus,lh


In [74]:
# UKBB_dkt_ct_fields = "/home/nikhil/projects/brain_changes/brain-diff/metadata/UKBB_DKT_CT_Fields.csv"
# ukbb_dkt_measure_df.to_csv(UKBB_dkt_ct_fields)